In [197]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten,Dense
import cv2
import os
import numpy as np

In [198]:
images = []
number_of_images = []
#load images
def load_images_from_folder(folders):
    count = 0
    for folder in folders:
        for filename in os.listdir(folder):
            img = cv2.imread(os.path.join(folder,filename))
            if img is not None:
                img = cv2.resize(img,(256,256))
                images.append(img)
                count = count + 1
        number_of_images.append(count)
        count = 0

In [199]:
foldersname = ['Cobra_Crop/Head','KingCobra_Crop/Head','Krait_Crop/Head']
load_images_from_folder(foldersname)

X = np.array(images) / 255  #ปรับค่าของตัวแปรให้อยู่ในช่วง 0-1 

In [200]:
X

array([[[[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]],

        [[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]],

        [[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]],

        ...,

        [[0.41176471, 0.36078431, 0.36470588],
         [0.41568627, 0.36470588, 0.36470588]

In [201]:
#เตรียม output
def defined_output(number_of_images):
    for count,num in enumerate(number_of_images):
        for i in range(num):
            classes = len(number_of_images)
            output = np.zeros((classes,), dtype=int)
            output[count] = 1
            y.append(output)

In [202]:
y = []
defined_output(number_of_images)
y = np.array(y)

In [203]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [205]:
from tensorflow.keras.models import Sequential , Model
from tensorflow.keras.layers import Dense , Activation
from tensorflow.keras.layers import Dropout , GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD , RMSprop , Adadelta , Adam
from tensorflow.keras.layers import Conv2D , BatchNormalization
from tensorflow.keras.layers import MaxPooling2D

In [206]:
model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(256,256,3))

x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax')(x)

In [207]:
model = Model(model.input, predictions)

In [208]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [209]:
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_1034 (Conv2D)            (None, 127, 127, 32) 864         input_12[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1034 (Batch (None, 127, 127, 32) 96          conv2d_1034[0][0]                
__________________________________________________________________________________________________
activation_1034 (Activation)    (None, 127, 127, 32) 0           batch_normalization_1034[0][0]   
___________________________________________________________________________________________

In [210]:
model.fit(X_train,y_train,batch_size=32,epochs=10,validation_split=0.3)

Epoch 1/10
2/2 [==============================] - 6s 3s/step - loss: 1.1758 - accuracy: 0.3830 - val_loss: 1.1699 - val_accuracy: 0.3333
Epoch 2/10
2/2 [==============================] - 5s 3s/step - loss: 1.0187 - accuracy: 0.4894 - val_loss: 1.1299 - val_accuracy: 0.4286
Epoch 3/10
2/2 [==============================] - 5s 3s/step - loss: 0.9268 - accuracy: 0.5957 - val_loss: 1.0838 - val_accuracy: 0.5238
Epoch 4/10
2/2 [==============================] - 5s 3s/step - loss: 0.8088 - accuracy: 0.7021 - val_loss: 1.0544 - val_accuracy: 0.5714
Epoch 5/10
2/2 [==============================] - 5s 3s/step - loss: 0.6750 - accuracy: 0.7660 - val_loss: 1.0179 - val_accuracy: 0.6190
Epoch 6/10
2/2 [==============================] - 5s 3s/step - loss: 0.6117 - accuracy: 0.7234 - val_loss: 0.9877 - val_accuracy: 0.6190
Epoch 7/10
2/2 [==============================] - 5s 3s/step - loss: 0.5756 - accuracy: 0.8085 - val_loss: 0.9644 - val_accuracy: 0.6667
Epoch 8/10
2/2 [=========================

In [211]:
model.predict(X_test)

array([[0.11226137, 0.7662324 , 0.12150628],
       [0.15931398, 0.09075771, 0.7499283 ],
       [0.2443828 , 0.36110163, 0.39451563],
       [0.30842596, 0.4158101 , 0.2757639 ],
       [0.07861356, 0.82118326, 0.1002031 ],
       [0.12241904, 0.7610191 , 0.11656182],
       [0.1910486 , 0.13418776, 0.6747637 ],
       [0.12255186, 0.64259857, 0.2348495 ],
       [0.0687211 , 0.78649235, 0.14478646],
       [0.23157711, 0.70685655, 0.06156628],
       [0.17049037, 0.4594865 , 0.37002313],
       [0.21936116, 0.45793688, 0.32270193],
       [0.30372128, 0.09143899, 0.60483974],
       [0.05392414, 0.09578867, 0.8502872 ],
       [0.15735857, 0.08179267, 0.7608487 ],
       [0.32863048, 0.38393128, 0.28743824],
       [0.10280983, 0.06509719, 0.832093  ],
       [0.15973565, 0.62648994, 0.21377444],
       [0.15164258, 0.52133155, 0.32702598],
       [0.22878228, 0.31678292, 0.45443484],
       [0.18721892, 0.06743734, 0.7453438 ],
       [0.20820531, 0.56575406, 0.2260406 ],
       [0.

In [212]:
model.evaluate(X_test,y_test)

2/2 [==============================] - 0s 51ms/step - loss: 0.9013 - accuracy: 0.5882


[0.9012659192085266, 0.5882353186607361]